# 34-傅立叶变换——高通滤波

将图像灰度化之后进行傅立叶变换并进行高通滤波，之后再用傅立叶逆变换复原

在这里，我们使用可以去除低频部分，只保留高频部分的高通滤波器。假设从低频的中心到高频的距离为$r$，我们保留$0.2\ r$​的低频分量。

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
# DFT hyper-parameters
K, L = 64, 64
channel = 3

In [3]:
# DFT
def dft(img):
    H, W, C = img.shape

    G = np.zeros((L, K, channel), dtype=np.complex)
    # np.complex创建一个值为 real+imag*j的复数或者转化为一个字符串或数为复数？？

    # 准备对应于原始图像位置的处理过的索引
    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # dft
    for c in range(channel):
            for l in range(L):
                for k in range(K):
                    G[l, k, c] = np.sum(img[..., c] * np.exp(-2j * np.pi * (x * k / K + y * l / L))) / np.sqrt(K * L)

    return G

In [4]:
# IDFT
def idft(G):
    H, W, _ = G.shape
    out = np.zeros((H, W, channel), dtype=np.float32)

    # 准备对应于原始图像位置的处理过的索引
    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # idft
    for c in range(channel):
            for l in range(H):
                for k in range(W):
                    out[l, k, c] = np.abs(np.sum(G[..., c] * np.exp(2j * np.pi * (x * k / W + y * l / H)))) / np.sqrt(W * H)

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

In [5]:
# HPF
def hpf(G, ratio=0.5):
    H, W, C = G.shape

    _G = np.zeros_like(G)
    _G[:H//2, :W//2] = G[H//2:, W//2:]
    _G[:H//2, W//2:] = G[H//2:, :W//2]
    _G[H//2:, :W//2] = G[:H//2, W//2:]
    _G[H//2:, W//2:] = G[:H//2, :W//2]

    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # 滤波器
    _x = x - W // 2
    _y = y - H // 2
    r = np.sqrt(_x ** 2 + _y ** 2)
    mask = np.ones((H, W), dtype=np.float32)
    mask[r < (W // 2 * ratio)] = 0

    mask = np.repeat(mask, channel).reshape(H, W, channel)

    # filtering
    _G *= mask

    G[:H//2, :W//2] = _G[H//2:, W//2:]
    G[:H//2, W//2:] = _G[H//2:, :W//2]
    G[H//2:, :W//2] = _G[:H//2, W//2:]
    G[H//2:, W//2:] = _G[:H//2, :W//2]

    return G

In [8]:
img = cv2.imread('../picture/chan_small.jpg').astype(np.float32)

In [9]:
img_ = dft(img)

In [10]:
img_hpf = hpf(img_)

In [11]:
img_hpf = idft(img_hpf)

In [12]:
cv2.imwrite('../picture/chan_result34_hpf.jpg', img_hpf)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_hpf)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 35-傅立叶变换——带通滤波

将图像灰度化之后进行傅立叶变换并进行带通滤波，之后再用傅立叶逆变换复原

在这里，我们使用可以保留介于低频成分和高频成分之间的分量的带通滤波器。在这里，我们使用可以去除低频部分，只保留高频部分的高通滤波器。假设从低频的中心到高频的距离为$r$，我们保留$0.1\ r$至$0.5\ r$的分量。

In [13]:
# BPF
def bpf(G, ratio1=0.2, ratio2=0.8):
    H, W, C = G.shape

    _G = np.zeros_like(G)
    _G[:H//2, :W//2] = G[H//2:, W//2:]
    _G[:H//2, W//2:] = G[H//2:, :W//2]
    _G[H//2:, :W//2] = G[:H//2, W//2:]
    _G[H//2:, W//2:] = G[:H//2, :W//2]

    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # 滤波器
    _x = x - W // 2
    _y = y - H // 2
    r = np.sqrt(_x ** 2 + _y ** 2)
    mask = np.ones((H, W), dtype=np.float32)
    mask[(r < (W // 2 * ratio1)) | (r > (W // 2 * ratio2))] = 0

    mask = np.repeat(mask, channel).reshape(H, W, channel)

    # filtering
    _G *= mask

    G[:H//2, :W//2] = _G[H//2:, W//2:]
    G[:H//2, W//2:] = _G[H//2:, :W//2]
    G[H//2:, :W//2] = _G[:H//2, W//2:]
    G[H//2:, W//2:] = _G[:H//2, :W//2]

    return G

In [14]:
img_ = bpf(img_)

In [15]:
img_bpf = idft(img_)

In [16]:
cv2.imwrite('../picture/chan_result35_bpf.jpg', img_bpf)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_bpf)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 36-JPEG 压缩——第一步：离散余弦变换（Discrete Cosine Transformation）

图像灰度化之后，先进行离散余弦变换，再进行离散余弦逆变换吧！

离散余弦变换（Discrete Cosine Transformation）是一种使用下面式子计算的频率变换： $$ 0\leq u,\ v\leq T\ F(u,v)=\frac{2}{T}\ C(u)\ C(v)\ \sum\limits_{y=0}^{T-1}\ \sum\limits_{x=0}^{T-1}\ I(x,y)\ \cos(\frac{(2\ x+1)\ u\ \pi}{2\ T}\ \cos(\frac{(2\ y+1)\ v\ \pi}{2\ T})\ C(u)= \begin{cases} \frac{1}{\sqrt{2}}& (\text{if}\ u=0)\ 1&(\text{else}) \end{cases} $$ 离散余弦逆变换（Inverse Discrete Cosine Transformation）是离散余弦变换的逆变换，使用下式定义。

在这里，$K$是决定图像复原时分辨率高低的参数。$K=T$时，DCT的系数全被保留，因此IDCT时分辨率最大。$K=1$或$K=2$时，图像复原时的信息量（DCT系数）减少，分辨率降低。如果适当地设定$K$，可以减小文件大小。 $$ 1\leq K\leq T\ f(x,y)=\frac{2}{T}\ \sum\limits_{u=0}^{K-1}\sum\limits_{v=0}^{K-1}\ C(u)\ C(v)\ F(u,v)\ \cos(\frac{(2\ x+1)\ u\ \pi}{2\ T})\ \cos(\frac{(2\ y+1)\ v\ \pi}{2\ T})\ C(u)= \begin{cases} \frac{1}{\sqrt{2}}& (\text{if}\ u=0)\ 1&(\text{else}) \end{cases} $$ 在这里我们先将图像分割成$8\times8$的小块，在各个小块中使用离散余弦变换编码，使用离散余弦逆变换解码，这就是 JPEG的编码过程。现在我们也同样地，把图像分割成$8\times8$的小块，然后进行离散余弦变换和离散余弦逆变换。

In [28]:
# DCT hyoer-parameter
T = 8
K = 8
channel = 3

In [18]:
# DCT weight
def w(x, y, u, v):
    cu = 1.
    cv = 1.
    if u == 0:
        cu /= np.sqrt(2)
    if v == 0:
        cv /= np.sqrt(2)
    theta = np.pi / (2 * T)
    
    return (( 2 * cu * cv / T) * np.cos((2*x+1)*u*theta) * np.cos((2*y+1)*v*theta))

In [19]:
# DCT
def dct(img):
    H, W, _ = img.shape

    F = np.zeros((H, W, channel), dtype=np.float32)

    for c in range(channel):
        for yi in range(0, H, T):
            for xi in range(0, W, T):
                for v in range(T):
                    for u in range(T):
                        for y in range(T):
                            for x in range(T):
                                F[v+yi, u+xi, c] += img[y+yi, x+xi, c] * w(x,y,u,v)

    return F

In [20]:
# IDCT
def idct(F):
    H, W, _ = F.shape

    out = np.zeros((H, W, channel), dtype=np.float32)

    for c in range(channel):
        for yi in range(0, H, T):
            for xi in range(0, W, T):
                for y in range(T):
                    for x in range(T):
                        for v in range(K):
                            for u in range(K):
                                out[y+yi, x+xi, c] += F[v+yi, u+xi, c] * w(x,y,u,v)

    out = np.clip(out, 0, 255)
    out = np.round(out).astype(np.uint8)

    return out

In [35]:
img = cv2.imread('../picture/chans.png').astype(np.float32)

In [36]:
img.shape

(128, 128, 3)

In [37]:
F = dct(img)

In [40]:
img_dct = idct(F)

In [41]:
cv2.imwrite("../picture/chan_result36_Frequency Domain.jpg", F)

True

In [42]:
cv2.imwrite('../picture/chan_result36_Discrete Cosine Transformation.jpg', img_dct)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_dct)
cv2.waitKey(0)
cv2.destroyAllWindows()